In [ ]:
# 1. 라이브러리 임포트 및 유틸 함수 정의
from glob import glob
import os
import openslide
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from skimage.transform import resize
from tqdm import tqdm
from pathlib import Path

In [ ]:
# 2. 데이터 준비 및 슬라이드 매칭 함수

def get_prefix(filename):
    basename = os.path.basename(filename)
    parts = basename.split('-')
    if len(parts) >= 4:
        return '-'.join(parts[:4])
    return basename

def match_slide_pairs(he_glob, pdl1_glob):
    he_slide_list = glob(he_glob)
    pdl1_slide_list = glob(pdl1_glob)
    he_prefixes = {get_prefix(f): f for f in he_slide_list}
    pdl1_prefixes = {get_prefix(f): f for f in pdl1_slide_list}
    common_prefixes = set(he_prefixes.keys()) & set(pdl1_prefixes.keys())
    matched_he_slides = [he_prefixes[prefix] for prefix in common_prefixes]
    matched_pdl1_slides = [pdl1_prefixes[prefix] for prefix in common_prefixes]
    return matched_he_slides, matched_pdl1_slides, sorted(common_prefixes)

# 사용 예시
he_glob = '../../data/IHC_HE_Pair_Data_GA_hospital/PD-L1(HnE)/*.ndpi'
pdl1_glob = '../../data/IHC_HE_Pair_Data_GA_hospital/PD-L1(22C3)/*.ndpi'
matched_he_slides, matched_pdl1_slides, common_prefixes = match_slide_pairs(he_glob, pdl1_glob)
print(f"공통 파일 개수: {len(common_prefixes)}")
print(f"HE slides: {len(matched_he_slides)}")
print(f"PD-L1 slides: {len(matched_pdl1_slides)}")
print(f"매칭된 prefix 예시: {common_prefixes[:5]}")

In [ ]:
# 3. 썸네일 및 마스크 생성 함수

def get_thumbnail_and_mask(slide_path, downsample=30):
    slide = openslide.OpenSlide(slide_path)
    thumb = slide.get_thumbnail((slide.dimensions[0]//downsample, slide.dimensions[1]//downsample))
    thumb_np = np.array(thumb)
    gray = cv2.cvtColor(thumb_np, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return slide, thumb_np, mask

# 예시: i번째 쌍
idx = 5
he_slide, he_thumb_np, he_mask = get_thumbnail_and_mask(matched_he_slides[idx])
pdl1_slide, pdl1_thumb_np, pdl1_mask = get_thumbnail_and_mask(matched_pdl1_slides[idx])
print(f"HE thumbnail shape: {he_thumb_np.shape}")
print(f"PDL1 thumbnail shape: {pdl1_thumb_np.shape}")
print(f"HE mask tissue ratio: {np.sum(he_mask > 0) / he_mask.size:.2%}")
print(f"PDL1 mask tissue ratio: {np.sum(pdl1_mask > 0) / pdl1_mask.size:.2%}")

In [ ]:
# Grayscale 변환
he_gray = cv2.cvtColor(he_thumb_np, cv2.COLOR_RGB2GRAY)
pdl1_gray = cv2.cvtColor(pdl1_thumb_np, cv2.COLOR_RGB2GRAY)

# Otsu threshold로 조직 영역 마스크 생성
_, he_mask = cv2.threshold(he_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
_, pdl1_mask = cv2.threshold(pdl1_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 모폴로지 연산으로 노이즈 제거
kernel = np.ones((5,5), np.uint8)
he_mask = cv2.morphologyEx(he_mask, cv2.MORPH_CLOSE, kernel)
he_mask = cv2.morphologyEx(he_mask, cv2.MORPH_OPEN, kernel)
pdl1_mask = cv2.morphologyEx(pdl1_mask, cv2.MORPH_CLOSE, kernel)
pdl1_mask = cv2.morphologyEx(pdl1_mask, cv2.MORPH_OPEN, kernel)

print(f"HE thumbnail shape: {he_thumb_np.shape}")
print(f"PDL1 thumbnail shape: {pdl1_thumb_np.shape}")
print(f"HE mask tissue ratio: {np.sum(he_mask > 0) / he_mask.size:.2%}")
print(f"PDL1 mask tissue ratio: {np.sum(pdl1_mask > 0) / pdl1_mask.size:.2%}")

In [ ]:
# 4. 마스크 기반 정합 함수 (Grid Search)

def compute_mask_similarity(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    if union == 0:
        return 0
    return intersection / union

def transform_and_match(source_mask, target_mask, angle, scale, tx, ty):
    h, w = source_mask.shape
    target_h, target_w = target_mask.shape
    rotated = rotate(source_mask, angle, reshape=False, order=0)
    new_h, new_w = int(h * scale), int(w * scale)
    scaled = cv2.resize(rotated.astype(np.uint8), (new_w, new_h), interpolation=cv2.INTER_NEAREST)
    result = np.zeros((target_h, target_w), dtype=np.uint8)
    start_y = (target_h - scaled.shape[0]) // 2 + ty
    start_x = (target_w - scaled.shape[1]) // 2 + tx
    src_start_y = max(0, -start_y)
    src_start_x = max(0, -start_x)
    src_end_y = min(scaled.shape[0], target_h - start_y)
    src_end_x = min(scaled.shape[1], target_w - start_x)
    dst_start_y = max(0, start_y)
    dst_start_x = max(0, start_x)
    dst_end_y = dst_start_y + (src_end_y - src_start_y)
    dst_end_x = dst_start_x + (src_end_x - src_start_x)
    if src_end_y > src_start_y and src_end_x > src_start_x:
        result[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = \
            scaled[src_start_y:src_end_y, src_start_x:src_end_x]
    similarity = compute_mask_similarity(result > 0, target_mask > 0)
    return similarity, result

In [ ]:
# 5. 정합 최적화 (Coarse-to-Fine Grid Search)

def optimize_mask_registration(he_mask, pdl1_mask):
    # coarse search
    angle_range_coarse = np.arange(-180, 181, 10)
    scale_range_coarse = np.arange(0.8, 1.21, 0.1)
    max_translation_coarse = int(min(he_mask.shape) * 0.1)
    translation_range_coarse = np.arange(-max_translation_coarse, max_translation_coarse + 1, max(1, max_translation_coarse // 2))
    best_score, best_angle, best_scale, best_tx, best_ty, best_transformed = 0, 0, 1.0, 0, 0, None
    results = []
    total_combinations = (len(angle_range_coarse) * len(scale_range_coarse) * len(translation_range_coarse) ** 2)
    with tqdm(total=total_combinations, desc="Coarse") as pbar:
        for angle in angle_range_coarse:
            for scale in scale_range_coarse:
                for tx in translation_range_coarse:
                    for ty in translation_range_coarse:
                        score, transformed = transform_and_match(he_mask, pdl1_mask, angle, scale, tx, ty)
                        results.append((angle, scale, tx, ty, score))
                        if score > best_score:
                            best_score, best_angle, best_scale, best_tx, best_ty, best_transformed = score, angle, scale, tx, ty, transformed.copy()
                        pbar.update(1)
    # fine search (angle/scale)
    angle_range_fine = np.arange(best_angle - 10, best_angle + 11, 1)
    scale_range_fine = np.arange(max(0.5, best_scale - 0.1), min(1.5, best_scale + 0.11), 0.01)
    with tqdm(total=len(angle_range_fine) * len(scale_range_fine), desc="Fine A/S") as pbar:
        for angle in angle_range_fine:
            for scale in scale_range_fine:
                score, transformed = transform_and_match(he_mask, pdl1_mask, angle, scale, best_tx, best_ty)
                results.append((angle, scale, best_tx, best_ty, score))
                if score > best_score:
                    best_score, best_angle, best_scale, best_transformed = score, angle, scale, transformed.copy()
                pbar.update(1)
    # fine search (translation)
    translation_range_fine_x = np.arange(best_tx - 10, best_tx + 11, 1)
    translation_range_fine_y = np.arange(best_ty - 10, best_ty + 11, 1)
    with tqdm(total=len(translation_range_fine_x) * len(translation_range_fine_y), desc="Fine Trans") as pbar:
        for tx in translation_range_fine_x:
            for ty in translation_range_fine_y:
                score, transformed = transform_and_match(he_mask, pdl1_mask, best_angle, best_scale, tx, ty)
                results.append((best_angle, best_scale, tx, ty, score))
                if score > best_score:
                    best_score, best_tx, best_ty, best_transformed = score, tx, ty, transformed.copy()
                pbar.update(1)
    return best_angle, best_scale, best_tx, best_ty, best_score, best_transformed, results

# 실행 예시
print("=== 마스크 정합 최적화 시작 ===")
best_angle, best_scale, best_tx, best_ty, best_score, best_transformed, results = optimize_mask_registration(he_mask, pdl1_mask)
print(f"최적 결과: angle={best_angle}°, scale={best_scale:.4f}, tx={best_tx}, ty={best_ty}, IoU={best_score:.4f}")

In [ ]:
# 6. 정합 결과 시각화 함수 (3x3, HE/PD-L1 원본 모두 표시, 비교행 제거)

def plot_registration_results(
    he_mask, pdl1_mask, best_transformed,
    he_thumb_np, pdl1_thumb_np, best_angle, best_scale, best_tx, best_ty, best_score
):
    fig, axes = plt.subplots(3, 3, figsize=(24, 18))
    # 1행: 마스크
    axes[0, 0].imshow(he_mask, cmap='gray')
    axes[0, 0].set_title('HE Mask (Original)')
    axes[0, 0].axis('off')
    axes[0, 1].imshow(pdl1_mask, cmap='gray')
    axes[0, 1].set_title('PD-L1 Mask (Target)')
    axes[0, 1].axis('off')
    axes[0, 2].imshow(best_transformed, cmap='gray')
    axes[0, 2].set_title(f'HE Mask Transformed\n(angle={best_angle}°, scale={best_scale:.2f}x, tx={best_tx}, ty={best_ty})')
    axes[0, 2].axis('off')
    # 2행: Overlay
    overlay = np.zeros((*pdl1_mask.shape, 3), dtype=np.uint8)
    overlay[best_transformed > 0] = [255, 0, 0]
    overlay[pdl1_mask > 0] = overlay[pdl1_mask > 0] + [0, 0, 255]
    axes[1, 0].imshow(he_thumb_np)
    axes[1, 0].set_title('HE Thumbnail (Original)')
    axes[1, 0].axis('off')
    axes[1, 1].imshow(pdl1_thumb_np)
    axes[1, 1].set_title('PD-L1 Thumbnail (IHC)')
    axes[1, 1].axis('off')
    axes[2, 0].imshow(overlay)
    axes[2, 0].set_title(f'Mask Overlay (IoU={best_score:.4f})\nRed=HE, Blue=PD-L1, Purple=Both')
    axes[2, 0].axis('off')
    # HE 썸네일 변환
    he_transformed_img = rotate(he_thumb_np, best_angle, reshape=False, order=1)
    new_h, new_w = int(he_thumb_np.shape[0] * best_scale), int(he_thumb_np.shape[1] * best_scale)
    he_transformed_img = cv2.resize(he_transformed_img, (new_w, new_h))
    target_h, target_w = pdl1_thumb_np.shape[:2]
    result_img = np.zeros((target_h, target_w, 3), dtype=np.uint8)
    start_y = (target_h - he_transformed_img.shape[0]) // 2 + best_ty
    start_x = (target_w - he_transformed_img.shape[1]) // 2 + best_tx
    src_start_y = max(0, -start_y)
    src_start_x = max(0, -start_x)
    src_end_y = min(he_transformed_img.shape[0], target_h - start_y)
    src_end_x = min(he_transformed_img.shape[1], target_w - start_x)
    dst_start_y = max(0, start_y)
    dst_start_x = max(0, start_x)
    dst_end_y = dst_start_y + (src_end_y - src_start_y)
    dst_end_x = dst_start_x + (src_end_x - src_start_x)
    if src_end_y > src_start_y and src_end_x > src_start_x:
        result_img[dst_start_y:dst_end_y, dst_start_x:dst_end_x] = he_transformed_img[src_start_y:src_end_y, src_start_x:src_end_x]
    he_resized = cv2.resize(he_thumb_np, (pdl1_thumb_np.shape[1], pdl1_thumb_np.shape[0]))
    alpha = 0.5
    original_overlay = cv2.addWeighted(he_resized, alpha, pdl1_thumb_np, 1-alpha, 0)
    axes[2, 1].imshow(original_overlay)
    axes[2, 1].set_title('Original Overlay (HE+PD-L1, Before)')
    axes[2, 1].axis('off')
    axes[1, 2].imshow(result_img)
    axes[1, 2].set_title('HE Image Transformed')
    axes[1, 2].axis('off')
    # 3행: HE/PD-L1 원본, 변환 후 Overlay
    axes[2, 0].imshow(he_thumb_np)
    axes[2, 0].set_title('HE Thumbnail (Original)')
    axes[2, 0].axis('off')
    axes[2, 1].imshow(pdl1_thumb_np)
    axes[2, 1].set_title('PD-L1 Thumbnail (IHC)')
    axes[2, 1].axis('off')
    img_overlay = cv2.addWeighted(result_img, alpha, pdl1_thumb_np, 1-alpha, 0)
    axes[2, 2].imshow(img_overlay)
    axes[2, 2].set_title('Registered Overlay (After)')
    axes[2, 2].axis('off')
    plt.tight_layout()
    plt.show()

# 사용 예시
plot_registration_results(
    he_mask, pdl1_mask, best_transformed,
    he_thumb_np, pdl1_thumb_np, best_angle, best_scale, best_tx, best_ty, best_score
)

In [ ]:
# 7. 변환 파라미터 계산 및 저장 함수

def get_transform_matrix(angle, scale, tx, ty, center_x, center_y):
    angle_rad = np.radians(angle)
    cos_a = np.cos(angle_rad) * scale
    sin_a = np.sin(angle_rad) * scale
    M = np.array([
        [cos_a, -sin_a, -center_x * cos_a + center_y * sin_a + center_x + tx],
        [sin_a, cos_a, -center_x * sin_a - center_y * cos_a + center_y + ty],
        [0, 0, 1]
    ])
    return M

def save_transformation_params(he_slide_path, pdl1_slide_path, he_slide, pdl1_slide,
                               he_thumb_shape, pdl1_thumb_shape,
                               angle, scale, tx, ty, score, output_dir='../../data/IHC_HE_Pair_Data_GA_hospital/registration_params'):
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    prefix = get_prefix(he_slide_path)
    output_file = output_dir / f"{prefix}_registration.json"
    downsample_x = he_slide.dimensions[0] / he_thumb_shape[1]
    downsample_y = he_slide.dimensions[1] / he_thumb_shape[0]
    tx_fullres = tx * downsample_x
    ty_fullres = ty * downsample_y
    he_center_x = he_thumb_shape[1] / 2
    he_center_y = he_thumb_shape[0] / 2
    M_thumbnail = get_transform_matrix(angle, scale, tx, ty, he_center_x, he_center_y)
    he_center_x_full = he_slide.dimensions[0] / 2
    he_center_y_full = he_slide.dimensions[1] / 2
    M_fullres = get_transform_matrix(angle, scale, tx_fullres, ty_fullres, he_center_x_full, he_center_y_full)
    data = {
        'files': {
            'he_slide': str(he_slide_path),
            'pdl1_slide': str(pdl1_slide_path),
            'prefix': prefix
        },
        'dimensions': {
            'he_full': {
                'width': he_slide.dimensions[0],
                'height': he_slide.dimensions[1]
            },
            'pdl1_full': {
                'width': pdl1_slide.dimensions[0],
                'height': pdl1_slide.dimensions[1]
            },
            'he_thumbnail': {
                'width': he_thumb_shape[1],
                'height': he_thumb_shape[0]
            },
            'pdl1_thumbnail': {
                'width': pdl1_thumb_shape[1],
                'height': pdl1_thumb_shape[0]
            }
        },
        'transformation': {
            'thumbnail': {
                'angle_degrees': float(angle),
                'scale': float(scale),
                'translation_x': float(tx),
                'translation_y': float(ty),
                'matrix': M_thumbnail[:2, :].tolist()
            },
            'fullres': {
                'angle_degrees': float(angle),
                'scale': float(scale),
                'translation_x': float(tx_fullres),
                'translation_y': float(ty_fullres),
                'matrix': M_fullres[:2, :].tolist()
            }
        },
        'downsampling': {
            'factor_x': float(downsample_x),
            'factor_y': float(downsample_y)
        },
        'quality': {
            'iou_score': float(score)
        }
    }
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Saved registration parameters to: {output_file}")
    return output_file

# 저장 예시
output_file = save_transformation_params(
    matched_he_slides[idx], matched_pdl1_slides[idx],
    he_slide, pdl1_slide,
    he_thumb_np.shape, pdl1_thumb_np.shape,
    best_angle, best_scale, best_tx, best_ty, best_score
)
print(f"Saved parameters for slide pair {idx}")

In [ ]:
# 변환 파라미터를 JSON으로 저장
import json
from pathlib import Path

def save_transformation_params(he_slide_path, pdl1_slide_path, he_slide, pdl1_slide,
                               he_thumb_shape, pdl1_thumb_shape,
                               angle, scale, tx, ty, score, output_dir='./registration_results'):
    """
    슬라이드 정합 파라미터를 JSON으로 저장
    """
    # 출력 디렉토리 생성
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    # 파일 이름 생성 (prefix 기반)
    prefix = get_prefix(he_slide_path)
    output_file = output_dir / f"{prefix}_registration.json"
    
    # Downsampling factor 계산
    downsample_x = he_slide.dimensions[0] / he_thumb_shape[1]
    downsample_y = he_slide.dimensions[1] / he_thumb_shape[0]
    
    # 전체 해상도에서의 translation 계산
    tx_fullres = tx * downsample_x
    ty_fullres = ty * downsample_y
    
    # 썸네일 변환 행렬
    he_center_x = he_thumb_shape[1] / 2
    he_center_y = he_thumb_shape[0] / 2
    M_thumbnail = get_transform_matrix(angle, scale, tx, ty, he_center_x, he_center_y)
    
    # 전체 해상도 변환 행렬
    he_center_x_full = he_slide.dimensions[0] / 2
    he_center_y_full = he_slide.dimensions[1] / 2
    M_fullres = get_transform_matrix(angle, scale, tx_fullres, ty_fullres, 
                                     he_center_x_full, he_center_y_full)
    
    # 저장할 데이터 구성
    data = {
        'files': {
            'he_slide': str(he_slide_path),
            'pdl1_slide': str(pdl1_slide_path),
            'prefix': prefix
        },
        'dimensions': {
            'he_full': {
                'width': he_slide.dimensions[0],
                'height': he_slide.dimensions[1]
            },
            'pdl1_full': {
                'width': pdl1_slide.dimensions[0],
                'height': pdl1_slide.dimensions[1]
            },
            'he_thumbnail': {
                'width': he_thumb_shape[1],
                'height': he_thumb_shape[0]
            },
            'pdl1_thumbnail': {
                'width': pdl1_thumb_shape[1],
                'height': pdl1_thumb_shape[0]
            }
        },
        'transformation': {
            'thumbnail': {
                'angle_degrees': float(angle),
                'scale': float(scale),
                'translation_x': float(tx),
                'translation_y': float(ty),
                'matrix': M_thumbnail[:2, :].tolist()  # 2x3 affine matrix
            },
            'fullres': {
                'angle_degrees': float(angle),
                'scale': float(scale),
                'translation_x': float(tx_fullres),
                'translation_y': float(ty_fullres),
                'matrix': M_fullres[:2, :].tolist()  # 2x3 affine matrix
            }
        },
        'downsampling': {
            'factor_x': float(downsample_x),
            'factor_y': float(downsample_y)
        },
        'quality': {
            'iou_score': float(score)
        }
    }
    
    # JSON 저장
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Saved registration parameters to: {output_file}")
    return output_file

# 현재 슬라이드 쌍 저장
output_file = save_transformation_params(
    matched_he_slides[i], matched_pdl1_slides[i],
    he_slide, pdl1_slide,
    he_thumb_np.shape, pdl1_thumb_np.shape,
    best_angle, best_scale, best_tx, best_ty, best_score
)

print(f"\nSaved parameters for slide pair {i}")

## 전체 파이프라인 요약

1. **슬라이드 쌍 매칭**: `match_slide_pairs()`
2. **썸네일/마스크 생성**: `get_thumbnail_and_mask()`
3. **마스크 정합 최적화**: `optimize_mask_registration()`
4. **시각화**: `plot_registration_results()`
5. **변환 파라미터 저장**: `save_transformation_params()`
6. **JSON 불러오기/좌표 변환**: `load_and_apply_transformation()`, `transform_patch_coordinates()`

---

### 예시 사용법

```python
# 1. 슬라이드 쌍 매칭
matched_he_slides, matched_pdl1_slides, common_prefixes = match_slide_pairs(he_glob, pdl1_glob)

# 2. 썸네일/마스크 생성
he_slide, he_thumb_np, he_mask = get_thumbnail_and_mask(matched_he_slides[idx])
pdl1_slide, pdl1_thumb_np, pdl1_mask = get_thumbnail_and_mask(matched_pdl1_slides[idx])

# 3. 정합 최적화
best_angle, best_scale, best_tx, best_ty, best_score, best_transformed, results = optimize_mask_registration(he_mask, pdl1_mask)

# 4. 시각화
plot_registration_results(he_mask, pdl1_mask, best_transformed, he_thumb_np, pdl1_thumb_np, best_angle, best_scale, best_tx, best_ty, best_score)

# 5. 파라미터 저장
output_file = save_transformation_params(...)

# 6. JSON 불러오기 및 좌표 변환
params = load_and_apply_transformation(output_file)
pdl1_x, pdl1_y, corners = transform_patch_coordinates(he_x, he_y, patch_size, params)
```

---

- 각 함수는 재사용이 쉽도록 설계되어 있습니다.
- 반복 작업, 여러 쌍 처리 등은 for loop로 쉽게 확장 가능합니다.
- 필요시 추가 함수화/클래스화도 가능합니다.

In [ ]:
# 8. JSON 불러오기 및 좌표 변환 함수

def load_and_apply_transformation(json_path):
    with open(json_path, 'r') as f:
        params = json.load(f)
    print(f"Loaded registration parameters from: {json_path}")
    print(f"  Prefix: {params['files']['prefix']}")
    print(f"  HE slide: {Path(params['files']['he_slide']).name}")
    print(f"  PDL1 slide: {Path(params['files']['pdl1_slide']).name}")
    print(f"  Transformation: angle={params['transformation']['fullres']['angle_degrees']}°, "
          f"scale={params['transformation']['fullres']['scale']:.2f}, "
          f"tx={params['transformation']['fullres']['translation_x']:.1f}, "
          f"ty={params['transformation']['fullres']['translation_y']:.1f}")
    print(f"  Quality: IoU={params['quality']['iou_score']:.4f}")
    return params

def transform_patch_coordinates(he_x, he_y, patch_size, params):
    M = np.array(params['transformation']['fullres']['matrix'])
    corners = np.array([
        [he_x, he_y, 1],
        [he_x + patch_size, he_y, 1],
        [he_x + patch_size, he_y + patch_size, 1],
        [he_x, he_y + patch_size, 1]
    ]).T
    transformed = M @ corners
    center_x = transformed[0].mean()
    center_y = transformed[1].mean()
    pdl1_x = int(center_x - patch_size / 2)
    pdl1_y = int(center_y - patch_size / 2)
    return pdl1_x, pdl1_y, transformed.T

# 사용 예시
print("\n=== Example: How to use saved parameters for patch extraction ===\n")
print("# 1. Load JSON parameters")
print("params = load_and_apply_transformation('registration_results/CODIPAI-XXXX_registration.json')")
print("")
print("# 2. Extract aligned patches")
print("he_x, he_y = 10000, 20000  # HE 패치 좌표")
print("patch_size = 512")
print("pdl1_x, pdl1_y, corners = transform_patch_coordinates(he_x, he_y, patch_size, params)")
print("")
print("# 3. Read patches from WSI")
print("he_patch = he_slide.read_region((he_x, he_y), 0, (patch_size, patch_size))")
print("pdl1_patch = pdl1_slide.read_region((pdl1_x, pdl1_y), 0, (patch_size, patch_size))")